ФОРМУЛИРОВКА ЗАДАНИЯ
Задание 2. SQL
2.1 Очень усердные ученики.

2.1.1 Условие

Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. Каждое такое маленькое задание называется "горошиной".

Назовём очень усердным учеником того пользователя, который хотя бы раз за текущий месяц правильно решил 20 горошин.

2.1.2 Задача
Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов.NB! Под усердным студентом мы понимаем студента, который правильно решил 20 задач за текущий месяц.

ПОНИМАНИЕ ЗАДАЧИ:
По условиям данной задачи необходимо найти учеников которые были очень усердны, в данном условии говорится это ученики решившие 20 задач правильно. В моем понимании ученики решившие 20 и более задач считаются очень усердными, так как те что в сумме решили только 20 задач и те что в сумме решили больше 20 все прошли порог 20 задач, а значит вторых стоит также считать очень усердными. Либо они могут быть к примеру "сверхусердными", но тогда необходимо дополнить условие задачи ограничить количество задач необходимых для того чтобы студент попадал в группу усердных и "сверхусердных". 

In [1]:
#импортируем нужные библиотеки
import pandahouse as ph
import pandas as pd

In [2]:
#объявляем параметры подключения
connection = dict(database='default',
                  host='https://clickhouse.lab.karpov.courses',
                  user='student',
                  password='dpo_python_2020')

In [3]:
#пишем запрос, и получаем данные из clickhouse в pandas dataframe
query = """
SELECT
    count(diligent_student) as count_diligent_student-- подсчитываем количество строк равняющихся количеству усердных учеников
FROM 
    (SELECT
        formatDateTime(timest, '%Y-%m') AS month, --отбираем месяц в формате год-месяц
        st_id AS diligent_student, -- отбираем id учеников
        COUNT(correct) AS number_of_solved_peas -- отбираем количество решенных задач
    FROM default.peas --берем данные из таблицы 
    WHERE correct = 1 and formatDateTime(timest, '%Y-%m') = (SELECT min(formatDateTime(timest, '%Y-%m')) FROM default.peas) -- прописываем условие чтобы из основной таблицы отфильровались нерешенные задачи и чтобы месяц был максимальным(то есть текущим)
    GROUP BY  st_id, month -- группируем по месяцу и id чтобы оставить только уникальные id
    HAVING COUNT(correct) >= 20 -- оставляем тех кто решил 20 задач и более (усердные ученики)
    ORDER BY number_of_solved_peas DESC -- сортируем по количеству решенных задач
    ) AS table_select
"""
# сохраняем результат запроса в переменную 
diligent_student = ph.read_clickhouse(query, connection=connection) 
# отображаем результаты запроса
diligent_student

,count_diligent_student
0,136


In [4]:
#выгружаем данные из pandas в excel при необходимости
diligent_student.to_excel("DATA_SQL_2_1.xlsx", sheet_name='diligent_student', index=False, encoding='UTF-8')

In [5]:
#выгружаем данные из pandas в csv при необходимости
diligent_student.to_csv("DATA_SQL_2_1.csv", index=False)

ФОРМУЛИРОВКА ЗАДАНИЯ
2.2 Оптимизация воронки

2.2.1 Условие

Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.

2.2.2 Задача
Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:

ARPU 
ARPAU 
CR в покупку 
СR активного пользователя в покупку 
CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике
ARPU считается относительно всех пользователей, попавших в группы.

Активным считается пользователь, за все время решивший больше 10 задач правильно в любых дисциплинах.

Активным по математике считается пользователь, за все время решивший 2 или больше задач правильно по математике.

ПОНИМАНИЕ ЗАДАЧИ:
Согласно условию необходимо посчитать 5 продуктовых метрик по имеюмщимся у нас данным, в моем понимании задачи состояли из того чтобы посчитать каждую из метрик в разрезе групп, а это означает что по каждой из групп необходимо было посчитать количество пользователей с определенным ключевым действием, выручку общую по группам, а также собрать все эти данные и посчитать продуктовые метрики по группам таким образом чтобы у нас на каждую метрику на выходе имелось одно значение. Таким образом в итоге у нас получилось бы по 5 значений на каждую группу.

Также стоит уточнить что фраза ARPU считается относительно всех пользователей, попавших в группы расценивалась как подсчет метрики по всем пользователям попавшим в группу (то есть ARPU подсчитывается отдельно для каждой группы, а не общий ARPU по всем пользователям обоих групп)

Еще небольшое уточнение согласно ARPAU: так как ARPAU (Average Revenue Per Active User) - средний платеж на всех активных пользователей, не было уточнено брать ли сюда активных - только тех кто решил более 10 задач в любых дисциплинах или тех кто выполнил любое активное действие (покупка, решенная задача и т.д.) было принято решение подсчитывать ARPAU по всем активным пользователям согласно условию решивших более 10 задач в любых дисциплинах.

Также стоит уточнить что в данном задании как и в первом просилось выбрузить все в одном запросе, что и было выполнено с использованием выгрузки через Jupiter notebook с использованием библиотеки pandahouse

   1. 'ARPU' - средний платеж на всех пользователей
   2. 'ARPAU' - средний платеж на всех активных пользователей
   3. 'CR_Users_UsersPay' - конверсия (в %) всех пользователей в тех кто купил курсы
   4. 'CR_ActiveUsers_ActiveUsersPay' - конверсия (в %) активных пользователей в активных пользователей которые купили курсы 
   5. 'CR_ActiveUsersMath_ActiveUsersMathPay' - конверсия (в %) активных пользователей по математике в активных пользователей которые купили курсы по математике
   
   
   
   1. Метрика ARPU расчитывалась как ARPU = выручка по группам / количество уникальных пользователей по группам
   2. Метрика ARPAU рассчитывалась как ARPAU = выручка по группам / количество уникальных активных пользоватлей по группам
   3. Метрика CR_Users_UsersPay рассчитывалась как CR_Users_UsersPay = количество уникальных пользователей совершивших покупку по группам / количество уникальных пользователей по группам
   4. Метрика CR_ActiveUsers_ActiveUsersPay рассчитывалась как CR_ActiveUsers_ActiveUsersPay = количество уникальных активных пользователей совершивших покупку по группам / количество уникальных активных пользователей по группам
   5. Метрика CR_ActiveUsersMath_ActiveUsersMathPay рассчитывалась как CR_ActiveUsersMath_ActiveUsersMathPay = количество уникальных активных пользователей по математике совершивших покупку курсов по математике по группам / количество уникальных активных пользователей по математике по группам
   

In [13]:
#пишем запрос, и получаем данные из clickhouse в pandas dataframe
query = """
-- создаем дополнительные main таблицы для работы: main_table_1 - это обьединение таблиц peas + studs с сохранением строк из таблицы peas; 
-- main_table_2 - это обьединение таблиц final_project_check + studs с сохранением строк из таблицы final_project_check
WITH main_table_1 as (SELECT a.st_id as st_id, a.correct as correct, a.subject as discipline, b.test_grp as test_grp FROM default.peas as a LEFT JOIN default.studs as b ON a.st_id = b.st_id), main_table_2 as (SELECT c.st_id as st_id, c.money as money, c.subject as course, d.test_grp as test_grp FROM default.final_project_check as c LEFT JOIN default.studs as d ON c.st_id = d.st_id)

SELECT
    one.test_grp as test_grp, -- отбираем группы пользователей
    (Revenue / users) as ARPU, -- подсчитываем и отбираем ARPU
    (Revenue / active_users) as ARPAU, -- подсчитываем и отбираем метрику ARPAU
    ((users_pay / users) * 100) as CR_Users_UsersPay, -- подсчитываем и отбираем CR в покупку
    ((active_users_pay / active_users) * 100) as CR_ActiveUsers_ActiveUsersPay, -- подсчитываем и отбираем CR активного пользователя в покупку
    ((active_users_math_pay / active_users_math) * 100) as CR_ActiveUsersMath_ActiveUsersMathPay -- подсчитываем и отбираем CR активного по дисциплине Math в покупку курса по Math
FROM
--подсчитываем общюю выручку по группам для подсчета метрики
    (SELECT test_grp, sum(w.money) as Revenue FROM (SELECT st_id, test_grp FROM main_table_1 GROUP BY test_grp, st_id) as q LEFT JOIN (SELECT st_id, sum(money) as money FROM main_table_2 GROUP BY st_id) as w ON q.st_id = w.st_id GROUP BY test_grp) as one
--обьединяем со следующей таблицей с подсчитанными исходными данными для подсчета метрики    
    INNER JOIN 
--подсчитываем количество уникальных пользователей по группам для подсчета метрики 
    (SELECT test_grp, count(DISTINCT st_id) as users from main_table_1 GROUP BY test_grp) as two ON one.test_grp = two.test_grp
--обьединяем со следующей таблицей с подсчитанными исходными данными для подсчета метрики    
    INNER JOIN
--подсчитываем количество уникальных пользователей которые произвели покупку курсов среди всех уникальных пользователей по группам для подсчета метрики
    (SELECT test_grp, count(DISTINCT r.st_id) as users_pay FROM (SELECT st_id, test_grp FROM main_table_1 GROUP BY test_grp, st_id) as r LEFT JOIN (SELECT st_id, sum(money) as money FROM main_table_2 GROUP BY st_id) as t ON r.st_id = t.st_id WHERE money > 0 GROUP BY test_grp) as three ON one.test_grp = three.test_grp
--обьединяем со следующей таблицей с подсчитанными исходными данными для подсчета метрики    
    INNER JOIN 
--подсчитываем количество уникальных активных пользователей по группам для подсчета метрики
    (SELECT test_grp, count(DISTINCT st_id) as active_users from (select test_grp, st_id, count(correct) from main_table_1 WHERE correct = 1 GROUP BY test_grp, st_id HAVING count(correct) > 10) as active_table GROUP BY test_grp) as four ON one.test_grp = four.test_grp
--обьединяем со следующей таблицей с подсчитанными исходными данными для подсчета метрики    
    INNER JOIN 
--подсчитываем количество уникальных активных пользователей которые произвели покупку из всех уникальных активных пользователей по группам для подсчета метрики
    (SELECT test_grp, count(DISTINCT y.st_id) as active_users_pay FROM (select test_grp, st_id, count(correct) from main_table_1 WHERE correct = 1 GROUP BY test_grp, st_id HAVING count(correct) > 10) as y LEFT JOIN (SELECT st_id, sum(money) as money FROM main_table_2 GROUP BY st_id) as u ON y.st_id = u.st_id WHERE money > 0 GROUP BY test_grp) as five ON one.test_grp = five.test_grp
--обьединяем со следующей таблицей с подсчитанными исходными данными для подсчета метрики    
    INNER JOIN 
--подсчитываем количество уникальных активных пользователей по дисциплине математика по группам для подсчета метрики 
    (SELECT test_grp, count(DISTINCT st_id) as active_users_math from (select test_grp, st_id, count(correct) from main_table_1 WHERE correct = 1 and discipline = 'Math' GROUP BY test_grp, st_id HAVING count(correct) >= 2) as active_math_table GROUP BY test_grp) as six ON one.test_grp = six.test_grp
--обьединяем со следующей таблицей с подсчитанными исходными данными для подсчета метрики    
    INNER JOIN 
--подсчитываем количество уникальных активных пользователей по дисциплине математика которые произвели покупку курса по математике среди всех уникальных активных пользователей по дисциплене математика по группам для подсчета метрики 
    (SELECT test_grp, count(DISTINCT i.st_id) as active_users_math_pay FROM (select test_grp, st_id, count(correct) from main_table_1 WHERE correct = 1 and discipline = 'Math' GROUP BY test_grp, st_id HAVING count(correct) >= 2) as i LEFT JOIN (SELECT st_id, sum(money) as money FROM main_table_2 WHERE course = 'Math' GROUP BY st_id) as o ON i.st_id = o.st_id WHERE money > 0 GROUP BY test_grp) as seven ON one.test_grp = seven.test_grp
"""
# сохраняем результат запроса в переменную 
funnel_optimization = ph.read_clickhouse(query, connection=connection)
# отображаем результаты запроса
funnel_optimization

,test_grp,ARPU,ARPAU,CR_Users_UsersPay,CR_ActiveUsers_ActiveUsersPay,CR_ActiveUsersMath_ActiveUsersMathPay
0,control,8393.939394,10905.511811,9.090909,11.023622,6.122449
1,pilot,22832.167832,34010.416667,20.979021,26.041667,9.523810


В случае если выдает верхний скрипт выдает ошибку 414 Client Error: Request-URI Too Large for url использовать скрипт ниже (без комментариев к действиям)

In [14]:
#пишем запрос, и получаем данные из clickhouse в pandas dataframe
query = """
WITH main_table_1 as (SELECT a.st_id as st_id, a.correct as correct, a.subject as discipline, b.test_grp as test_grp FROM default.peas as a LEFT JOIN default.studs as b ON a.st_id = b.st_id), main_table_2 as (SELECT c.st_id as st_id, c.money as money, c.subject as course, d.test_grp as test_grp FROM default.final_project_check as c LEFT JOIN default.studs as d ON c.st_id = d.st_id)

SELECT 
    one.test_grp as test_grp,
    (Revenue / users) as ARPU,
    (Revenue / active_users) as ARPAU,
    ((users_pay / users) * 100) as CR_Users_UsersPay,
    ((active_users_pay / active_users) * 100) as CR_ActiveUsers_ActiveUsersPay,
    ((active_users_math_pay / active_users_math) * 100) as CR_ActiveUsersMath_ActiveUsersMathPay
FROM
    (SELECT test_grp, sum(w.money) as Revenue FROM (SELECT st_id, test_grp FROM main_table_1 GROUP BY test_grp, st_id) as q LEFT JOIN (SELECT st_id, sum(money) as money FROM main_table_2 GROUP BY st_id) as w ON q.st_id = w.st_id GROUP BY test_grp) as one
    INNER JOIN 
    (SELECT test_grp, count(DISTINCT st_id) as users from main_table_1 GROUP BY test_grp) as two ON one.test_grp = two.test_grp
    INNER JOIN
    (SELECT test_grp, count(DISTINCT r.st_id) as users_pay FROM (SELECT st_id, test_grp FROM main_table_1 GROUP BY test_grp, st_id) as r LEFT JOIN (SELECT st_id, sum(money) as money FROM main_table_2 GROUP BY st_id) as t ON r.st_id = t.st_id WHERE money > 0 GROUP BY test_grp) as three ON one.test_grp = three.test_grp
    INNER JOIN 
    (SELECT test_grp, count(DISTINCT st_id) as active_users from (select test_grp, st_id, count(correct) from main_table_1 WHERE correct = 1 GROUP BY test_grp, st_id HAVING count(correct) > 10) as active_table GROUP BY test_grp) as four ON one.test_grp = four.test_grp
    INNER JOIN 
    (SELECT test_grp, count(DISTINCT y.st_id) as active_users_pay FROM (select test_grp, st_id, count(correct) from main_table_1 WHERE correct = 1 GROUP BY test_grp, st_id HAVING count(correct) > 10) as y LEFT JOIN (SELECT st_id, sum(money) as money FROM main_table_2 GROUP BY st_id) as u ON y.st_id = u.st_id WHERE money > 0 GROUP BY test_grp) as five ON one.test_grp = five.test_grp
    INNER JOIN 
    (SELECT test_grp, count(DISTINCT st_id) as active_users_math from (select test_grp, st_id, count(correct) from main_table_1 WHERE correct = 1 and discipline = 'Math' GROUP BY test_grp, st_id HAVING count(correct) >= 2) as active_math_table GROUP BY test_grp) as six ON one.test_grp = six.test_grp
    INNER JOIN 
    (SELECT test_grp, count(DISTINCT i.st_id) as active_users_math_pay FROM (select test_grp, st_id, count(correct) from main_table_1 WHERE correct = 1 and discipline = 'Math' GROUP BY test_grp, st_id HAVING count(correct) >= 2) as i LEFT JOIN (SELECT st_id, sum(money) as money FROM main_table_2 WHERE course = 'Math' GROUP BY st_id) as o ON i.st_id = o.st_id WHERE money > 0 GROUP BY test_grp) as seven ON one.test_grp = seven.test_grp
"""
# сохраняем результат запроса в переменную 
funnel_optimization = ph.read_clickhouse(query, connection=connection)
# отображаем результаты запроса
funnel_optimization

,test_grp,ARPU,ARPAU,CR_Users_UsersPay,CR_ActiveUsers_ActiveUsersPay,CR_ActiveUsersMath_ActiveUsersMathPay
0,control,8393.939394,10905.511811,9.090909,11.023622,6.122449
1,pilot,22832.167832,34010.416667,20.979021,26.041667,9.523810


In [10]:
#выгружаем данные из pandas в excel при необходимости
funnel_optimization.to_excel("DATA_SQL_2_2.xlsx", sheet_name='diligent_student', index=False, encoding='UTF-8')

In [11]:
#выгружаем данные из pandas в csv при необходимости
funnel_optimization.to_csv("DATA_SQL_2_2.csv", index=False)